In [2]:
from google.colab import auth
auth.authenticate_user()

# bibliotecas
from google.cloud import storage
import pandas as pd
import io
import json

# cliente de almacenamiento
client = storage.Client()

# bucket y archivo
bucket_name = 'pf_datawarehouse'
file_name = 'tip.json'

# obtener el bucket
bucket = client.get_bucket(bucket_name)

# obtener el blob (archivo)
blob = bucket.blob(file_name)

# descargar el contenido del archivo como string
file_content = blob.download_as_string()

# Convertir el contenido descargado de bytes a string
file_content = file_content.decode('utf-8')

# Dividir el contenido del archivo en múltiples objetos JSON
json_strings = file_content.split('\n')

# Filtrar y cargar cada objeto JSON individualmente
json_objects = [json.loads(json_str) for json_str in json_strings if json_str.strip()]

# Convertir la lista de objetos JSON en un DataFrame de pandas
df = pd.DataFrame(json_objects)

# Mostrar el DataFrame
df.head()


,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [3]:
df.shape

(908915, 5)

In [4]:
df_new = df.drop(columns=['date','compliment_count'])
df_new.head()

,user_id,business_id,text
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch


In [5]:
df_new['platform'] = 'Yelp'
df_new.head()

,user_id,business_id,text,platform
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,Yelp
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,Yelp
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,Yelp
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,Yelp
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,Yelp


In [6]:
duplicate_rows = df_new[df_new.duplicated()]

if len(duplicate_rows) > 0:
  print(f"Hay {len(duplicate_rows)} filas duplicadas en el DataFrame.")
else:
  print("No hay duplicadas")

Hay 2159 filas duplicadas en el DataFrame.


In [7]:
df_new = df_new.drop_duplicates()

In [8]:
# prompt: exportar la tabla al bucket etl_archivos

# definir el nombre del archivo a exportar
file_name = 'tipsYelp.csv'

# convertir el DataFrame a CSV
csv_data = df_new.to_csv(index=False)

# crear un nuevo blob en el bucket de destino
blob = client.bucket('etl_archivos').blob(file_name)

# subir el contenido del DataFrame al blob
blob.upload_from_string(csv_data)

# imprimir un mensaje de confirmación
print(f"El archivo '{file_name}' ha sido exportado al bucket 'etl_archivos'.")


El archivo 'tipsYelp.csv' ha sido exportado al bucket 'etl_archivos'.


In [9]:
# cliente de almacenamiento
client = storage.Client()

# bucket y archivo
bucket_name = 'etl_archivos'
file_name = 'tipsYelp.csv'

# obtener el bucket
bucket = client.get_bucket(bucket_name)

# obtener el blob (archivo)
blob = bucket.blob(file_name)

# descargar el contenido del archivo como string
file_content = blob.download_as_string()

# leer el contenido como DataFrame
df = pd.read_csv(io.BytesIO(file_content))

# mostrar el DataFrame
df.head()

,user_id,business_id,text,platform
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,Yelp
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,Yelp
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,Yelp
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,Yelp
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,Yelp
